# Підготовка та Аналіз даних
## Лабораторна робота №2
## Наука про дані: підготовчий етап
<b>Мета роботи</b>: ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки,
зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції.
#### ФБ-35 Тихомирова Софія

In [1]:
import urllib.request  # For downloading data from URLs
import pandas as pd    # For working with dataframes
from datetime import datetime  # For handling date and time
import os              # For interacting with the file system

In [ ]:
Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. 
Ця процедура має бути  автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його 
імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, довантаження нових даних та колізію даних.

In [24]:
def download_vhi_data(province_id, output_dir):
    """
    Downloads VHI data for a given province ID and saves it with a timestamped filename.
    :param province_id: The ID of the administrative region (1-25).
    :param output_dir: Directory where the downloaded files will be saved.
    """
    # Define the URL template
    url_template = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean"
    
    # Format the URL with the provided province ID
    url = url_template.format(province_id)
    
    # Get the current date and time for the filename
    now = datetime.now()
    timestamp = now.strftime("%d%m%Y%H%M%S")  # Format: DDMMYYYYHHMMSS
    
    # Define the output filename
    filename = f"NOAA_ID{province_id}_{timestamp}.csv"
    filepath = os.path.join(output_dir, filename)
    
    # Download the file
    try:
        with urllib.request.urlopen(url) as response:
            data = response.read()  # Read the content of the URL
            
            # Check if the downloaded data is empty
            if not data:
                print(f"No data received for province ID {province_id}")
                return
            
            # Save the data to the file
            with open(filepath, 'wb') as out_file:
                out_file.write(data)
                
            print(f"File successfully saved: {filepath}")
    except Exception as e:
        print(f"Error downloading data for province ID {province_id}: {e}")

In [ ]:
Зчитати завантажені текстові файли у фрейм (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). 
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не  повинно бути спеціалізованих символів, пробілів тощо).
Ця задача має бути  реалізована у вигляді окремої процедури, яка на вхід приймає шлях до  директорії, в якій зберігаються файли.

In [40]:
def read_files_to_dataframe(directory):
    """
    Reads all CSV files in the specified directory into a single dataframe.
    :param directory: Directory containing the CSV files.
    :return: A pandas dataframe with meaningful column names and a 'Province_ID' column.
    """
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']  # Define column headers
    all_data = []  # List to store individual dataframes
    
    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            
            # Extract province ID from the filename (e.g., "NOAA_ID1_20231010.csv")
            try:
                # Split the filename and extract the numeric part of the province ID
                province_id_str = filename.split('_')[1]  # e.g., 'ID10'
                province_id = int(province_id_str[2:])   # Remove 'ID' prefix and convert to int
            except (ValueError, IndexError):
                print(f"Skipping file due to invalid filename format: {filename}")
                continue
            
            # Read the file into a dataframe
            try:
                df = pd.read_csv(filepath, header=1, names=headers)
                
                # Drop rows with invalid VHI values (-1)
                df = df[df['VHI'] != -1]
                
                # Convert 'Year' column to integer
                df['Year'] = pd.to_numeric(df['Year'], errors='coerce')  # Convert to numeric, handle errors
                
                # Add a 'Province_ID' column
                df['Province_ID'] = province_id
                
                # Add the dataframe to the list
                all_data.append(df)
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
    
    # Combine all dataframes into one
    if not all_data:
        raise ValueError("No valid dataframes found in the directory.")
    
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

In [ ]:
Реалізувати окрему процедуру, яка змінить індекси областей, які використані на  порталі NOAA (за англійською абеткою) 
на наступні, за українською (виключно  старі індекси на нові):

In [41]:
def replace_province_ids(df):
    """
    Replaces NOAA province IDs with Ukrainian province IDs.
    :param df: Input dataframe with NOAA province IDs.
    :return: Dataframe with updated province IDs.
    """
    # Mapping of NOAA IDs to Ukrainian IDs
    id_mapping = {
        1: 22,   # Cherkasy → 22
        2: 24,   # Chernihiv → 24
        3: 23,   # Chernivtsi → 23
        4: 25,   # Crimea → 25
        5: 3,    # Dnipropetrovsk → 3
        6: 4,    # Donetsk → 4
        7: 8,    # Ivano-Frankivsk → 8
        8: 19,   # Kharkiv → 19
        9: 20,   # Kherson → 20
        10: 21,  # Khmelnytskyi → 21
        11: 10,  # Kirovohrad → 10
        12: 9,   # Kyiv → 9
        13: 11,  # Luhansk → 11
        14: 12,  # Lviv → 12
        15: 13,  # Mykolaiv → 13
        16: 14,  # Odessa → 14
        17: 15,  # Poltava → 15
        18: 16,  # Rivne → 16
        19: 17,  # Sumy → 17
        20: 18,  # Ternopil → 18
        21: 1,   # Vinnytsia → 1
        22: 2,   # Volyn → 2
        23: 6,   # Zakarpattia → 6
        24: 7,   # Zaporizhzhia → 7
        25: 5    # Zhytomyr → 5
    }
    
    # Replace IDs in the dataframe
    df['Province_ID'] = df['Province_ID'].map(id_mapping)
    return df

In [ ]:
Ряд VHI для області за вказаний рік.

In [35]:
def get_vhi_for_year(df, province_id, year):
    """
    Retrieves VHI series for a specific province and year.
    :param df: Input dataframe.
    :param province_id: ID of the province.
    :param year: Year of interest.
    :return: Filtered dataframe.
    """
    filtered_df = df[(df['Province_ID'] == province_id) & (df['Year'] == year)]
    return filtered_df

In [ ]:
Пошук екстремумів (min та max) для вказаних областей та років,  середнього, медіани.

In [36]:
def find_extremes(df, province_id, year):
    """
    Finds min, max, mean, and median VHI values for a specific province and year.
    :param df: Input dataframe.
    :param province_id: ID of the province.
    :param year: Year of interest.
    :return: Dictionary with statistical results.
    """
    filtered_df = df[(df['Province_ID'] == province_id) & (df['Year'] == year)]
    stats = {
        'Min': filtered_df['VHI'].min(),
        'Max': filtered_df['VHI'].max(),
        'Mean': filtered_df['VHI'].mean(),
        'Median': filtered_df['VHI'].median()
    }
    return stats

In [ ]:
Ряд VHI за вказаний діапазон років для вказаних областей.

In [37]:
def get_vhi_for_years(df, province_id, start_year, end_year):
    """
    Retrieves VHI series for a specific province and range of years.
    :param df: Input dataframe.
    :param province_id: ID of the province.
    :param start_year: Start year.
    :param end_year: End year.
    :return: Filtered dataframe.
    """
    filtered_df = df[(df['Province_ID'] == province_id) & (df['Year'] >= start_year) & (df['Year'] <= end_year)]
    return filtered_df

In [ ]:
Для всього набору даних виявити роки, протягом яких екстремальні 
посухи торкнулися більше вказаного відсотка областей по Україні.

In [38]:
def identify_drought_years(df, threshold_percentage):
    """
    Identifies years with extreme droughts affecting more than the threshold percentage of provinces.
    :param df: Input dataframe.
    :param threshold_percentage: Threshold percentage of affected provinces.
    :return: List of years and affected provinces.
    """
    total_provinces = 25
    threshold_count = int(total_provinces * threshold_percentage / 100)
    
    drought_years = []
    for year in df['Year'].unique():
        year_df = df[df['Year'] == year]
        extreme_provinces = year_df[year_df['VHI'] < 15]['Province_ID'].nunique()
        
        if extreme_provinces > threshold_count:
            drought_years.append({
                'Year': year,
                'Affected_Provinces': extreme_provinces,
                'VHI_Values': year_df[year_df['VHI'] < 15][['Province_ID', 'VHI']]
            })
    
    return drought_years

In [42]:
if __name__ == "__main__":
    # Define output directory
    output_dir = "vhi_data"
    os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist
    
    # Download data for all provinces
    for province_id in range(1, 26):
        download_vhi_data(province_id, output_dir)
    
    # Read all files into a dataframe
    try:
        df = read_files_to_dataframe(output_dir)
    except ValueError as e:
        print(e)
        exit()
    
    # Replace province IDs
    df = replace_province_ids(df)
    
    # Filter out rows with missing or invalid Year values
    df = df.dropna(subset=['Year'])
    
    # Example analyses
    print("VHI for Vinnytsia (ID=1) in 2020:")
    print(get_vhi_for_year(df, 1, 2020))
    
    print("\nExtremes for Kyiv (ID=9) in 2019:")
    print(find_extremes(df, 9, 2019))
    
    print("\nVHI for Lviv (ID=12) from 2015 to 2020:")
    print(get_vhi_for_years(df, 12, 2015, 2020))
    
    print("\nYears with extreme droughts affecting >20% of provinces:")
    print(identify_drought_years(df, 20))

File successfully saved: vhi_data\NOAA_ID1_10032025173501.csv
File successfully saved: vhi_data\NOAA_ID2_10032025173502.csv
File successfully saved: vhi_data\NOAA_ID3_10032025173503.csv
File successfully saved: vhi_data\NOAA_ID4_10032025173504.csv
File successfully saved: vhi_data\NOAA_ID5_10032025173505.csv
File successfully saved: vhi_data\NOAA_ID6_10032025173506.csv
File successfully saved: vhi_data\NOAA_ID7_10032025173506.csv
File successfully saved: vhi_data\NOAA_ID8_10032025173507.csv
File successfully saved: vhi_data\NOAA_ID9_10032025173508.csv
File successfully saved: vhi_data\NOAA_ID10_10032025173509.csv
File successfully saved: vhi_data\NOAA_ID11_10032025173510.csv
File successfully saved: vhi_data\NOAA_ID12_10032025173511.csv
File successfully saved: vhi_data\NOAA_ID13_10032025173512.csv
File successfully saved: vhi_data\NOAA_ID14_10032025173513.csv
File successfully saved: vhi_data\NOAA_ID15_10032025173514.csv
File successfully saved: vhi_data\NOAA_ID16_10032025173515.csv
F